In [34]:
import geopandas as gpd
import pandas as pd
import numpy as np 
import shapely

import os

os.chdir('C:\\PhD\\DissolutionProgramming\\REB---Rebellion-Paper\\')
RAW = 'Data\\Raw'
PROCESSED = 'Data\\Processed'


In [35]:
county_pop = pd.read_csv(os.path.join(RAW, 'CSV', 'county_pop_1600.csv'))
county_pop['county'] = county_pop['county'].str.upper()
houses = gpd.read_file(os.path.join(RAW, 'GIS', 'BNG Projections', 'allHousesBNG.shp'))

north_counties = [
    'NORTHUMBERLAND', 'CUMBERLAND', 'DURHAM', 'WESTMORLAND',
    'YORKSHIRE, NORTH RIDING', 'LANCASHIRE', 'YORKSHIRE, WEST RIDING',
    'YORKSHIRE, EAST RIDING', 'LINCOLNSHIRE'
]



In [36]:
# Sum together the netInc figures for the houses based on county
house_county_sum = houses.groupby('county')['netInc'].sum().reset_index()
# Merge the summed netInc with the county population data
merged_data = pd.merge(county_pop, house_county_sum, on='county', how='left')
# Calculate the per capita net income for each county
merged_data['net_inc_pc'] = merged_data['netInc'] / merged_data['pop_1600']

In [37]:
# Calculate net income per capita: North vs Rest of England
merged_data['region'] = merged_data['county'].apply(lambda x: 'North' if x in north_counties else 'Rest of England')

# Group by region and sum total netInc and population
regional_stats = merged_data.groupby('region').agg({
    'netInc': 'sum',
    'pop_1600': 'sum'
}).reset_index()

# Calculate per capita for each region
regional_stats['net_inc_pc'] = regional_stats['netInc'] / regional_stats['pop_1600']

print("Net Income Per Capita by Region:")
print(regional_stats[['region', 'netInc', 'pop_1600', 'net_inc_pc']])
print()

Net Income Per Capita by Region:
            region    netInc  pop_1600  net_inc_pc
0            North   26974.0    987727    0.027309
1  Rest of England  113121.0   3127163    0.036174



In [38]:
# Calculate mean and median religious house net income by county
house_county_stats = houses.groupby('county')['netInc'].agg(['mean', 'median', 'count']).reset_index()
house_county_stats.columns = ['county', 'mean_netInc', 'median_netInc', 'num_houses']

# Merge with regional classification
house_county_stats['region'] = house_county_stats['county'].apply(lambda x: 'North' if x in north_counties else 'Rest of England')

print("Mean and Median Net Income by County:")
print(house_county_stats[['county', 'region', 'mean_netInc', 'median_netInc', 'num_houses']].sort_values('median_netInc', ascending=False))

Mean and Median Net Income by County:
                     county           region  mean_netInc  median_netInc  \
32                   SURREY  Rest of England   421.909091          464.0   
21                MIDDLESEX  Rest of England   809.071429          337.5   
12          GLOUCESTERSHIRE  Rest of England   512.142857          247.0   
0              BEDFORDSHIRE  Rest of England   215.272727          241.0   
29                 SOMERSET  Rest of England   390.684211          215.0   
9                    DORSET  Rest of England   301.111111          214.0   
13                HAMPSHIRE  Rest of England   323.750000          214.0   
4                  CHESHIRE  Rest of England   347.000000          202.5   
8                     DEVON  Rest of England   275.000000          197.0   
36                WILTSHIRE  Rest of England   250.533333          177.0   
5                  CORNWALL  Rest of England   177.000000          175.0   
25          NOTTINGHAMSHIRE  Rest of England   166

In [39]:
# Calculate mean and median religious house net income by region
houses['region'] = houses['county'].apply(lambda x: 'North' if x in north_counties else 'Rest of England')

regional_house_stats = houses.groupby('region')['netInc'].agg(['mean', 'median', 'count']).reset_index()
regional_house_stats.columns = ['region', 'mean_netInc', 'median_netInc', 'num_houses']

print("Mean and Median Religious House Net Income by Region:")
print(regional_house_stats)
print()

Mean and Median Religious House Net Income by Region:
            region  mean_netInc  median_netInc  num_houses
0            North   164.475610           89.0         164
1  Rest of England   266.167059          124.0         425



In [40]:
# %% Religious Houses Per Capita Analysis
# Count religious houses by county
house_count_by_county = houses.groupby('county').size().reset_index(name='num_houses')

# Merge with population data
houses_pop_merged = pd.merge(county_pop, house_count_by_county, on='county', how='left')

# Fill NaN values with 0 (counties with no houses)
houses_pop_merged['num_houses'] = houses_pop_merged['num_houses'].fillna(0)

# Calculate houses per capita (per 10,000 people for readability)
houses_pop_merged['houses_per_10k'] = (houses_pop_merged['num_houses'] / houses_pop_merged['pop_1600']) * 10000

print("Religious Houses Per 10,000 People by County:")
print(houses_pop_merged[['county', 'pop_1600', 'num_houses', 'houses_per_10k']].sort_values('houses_per_10k', ascending=False))
print()

Religious Houses Per 10,000 People by County:
                     county  pop_1600  num_houses  houses_per_10k
38   YORKSHIRE, EAST RIDING     66520          31        4.660253
20             LINCOLNSHIRE    173199          71        4.099331
34             WARWICKSHIRE     65455          25        3.819418
22                  NORFOLK    171163          59        3.447007
26              OXFORDSHIRE     66909          22        3.288048
39  YORKSHIRE, NORTH RIDING    101596          32        3.149730
16          HUNTINGDONSHIRE     27627           8        2.895718
0              BEDFORDSHIRE     43059          12        2.786874
24           NORTHUMBERLAND     72923          20        2.742619
19           LEICESTERSHIRE     63140          17        2.692430
28               SHROPSHIRE     78958          21        2.659642
3            CAMBRIDGESHIRE     72492          18        2.483033
31                  SUFFOLK    138295          32        2.313894
15            HERTFORDSHIRE   

In [41]:
# Calculate houses per capita: North vs Rest of England
houses_pop_merged['region'] = houses_pop_merged['county'].apply(lambda x: 'North' if x in north_counties else 'Rest of England')

# Group by region and aggregate
regional_houses_stats = houses_pop_merged.groupby('region').agg({
    'num_houses': 'sum',
    'pop_1600': 'sum'
}).reset_index()

# Calculate houses per 10,000 people for each region
regional_houses_stats['houses_per_10k'] = (regional_houses_stats['num_houses'] / regional_houses_stats['pop_1600']) * 10000

print("Religious Houses Per 10,000 People by Region:")
print(regional_houses_stats[['region', 'num_houses', 'pop_1600', 'houses_per_10k']])
print()

# Calculate ratio of North to Rest of England
north_houses = regional_houses_stats[regional_houses_stats['region'] == 'North'].iloc[0]
rest_houses = regional_houses_stats[regional_houses_stats['region'] == 'Rest of England'].iloc[0]

print(f"North vs Rest of England Ratios:")
print(f"Houses Per Capita - North/Rest: {north_houses['houses_per_10k'] / rest_houses['houses_per_10k']:.3f}")

Religious Houses Per 10,000 People by Region:
            region  num_houses  pop_1600  houses_per_10k
0            North         216    987727        2.186839
1  Rest of England         581   3127163        1.857914

North vs Rest of England Ratios:
Houses Per Capita - North/Rest: 1.177
